In [ ]:
#painting , remove painting and predict words and numbers

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np
import cv2
import mediapipe as mp
import pytesseract as pyt
import time
mp_drawing = mp.solutions.drawing_utils
hand = mp.solutions.hands
mp_hands = mp.solutions.hands.Hands(False,1,0.7,0.5)
pyt.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"

fill=[]
pTime=0
paint=False
remove=False
word=False

In [3]:
cap=cv2.VideoCapture(0)
cap.set(3,1920)
cap.set(4,1080)
while True:
    ret,frame=cap.read()
    if ret==False :
        break
    rows, cols, _ = frame.shape
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = mp_hands.process(frame)
    
    cv2.circle(frame,(225,25),20,(255,0,0),-1)
    cv2.circle(frame,(425,25),20,(0,255,0),-1)
    cv2.circle(frame,(625,25),20,(0,0,255),-1)
            
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks( frame, hand_landmarks,hand.HAND_CONNECTIONS)
            x = [ int(landmark.x * cols) for landmark in hand_landmarks.landmark]
            y = [ int(landmark.y * rows) for landmark in hand_landmarks.landmark]
            
            if y[8] in [20,21,22,23,24,25,26,27,28,29,30] and x[8] in [220,221,222,223,224,225,226,227,228,229,230]:
                paint=True
                remove=False
                word=False
            
            elif y[8] in [20,21,22,23,24,25,26,27,28,29,30] and x[8] in [420,421,422,423,424,425,426,427,428,429,430]:
                paint=False
                remove=True
                word=False
            
            elif y[8] in [20,21,22,23,24,25,26,27,28,29,30] and x[8] in [620,621,622,623,624,625,626,627,628,629,630]:
                paint=False
                remove=False
                word=True
                
            if paint and y[12] > y[8] :
                fill.append([x[8],y[8]])
                
            elif remove and y[12] > y[8] :
                fill1=fill.copy()
                fill=[]
                for a,s in fill1 : 
                    if a+7 >x[8]>a-7  and s+7>y[8]>s-7 :
                        continue
                    else :
                        fill.append([a,s])

            
                
    if word :
        img=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
        boxes=pyt.image_to_data(img,lang='eng')
        for z,b in enumerate(boxes.splitlines()) :
            if z!=0 :
                b=b.split()
                if len(b)==12 :
                    fx,fy,fw,fh =int(b[6]),int(b[7]),int(b[8]),int(b[9])
                    cv2.rectangle(frame,(fx,fy),(fx+fw,fy+fh),(0,255,0),2)
                    cv2.putText(frame,b[11],(fx,fy),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
            
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    for x,y in fill :
        cv2.circle(frame,(x,y),10,(0,0,255),-1)
    

    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    
    cv2.imshow('MediaPipe Hands', frame)
    if cv2.waitKey(10) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()